# Loss Functions Implementation

In [2]:
import torch
from torch import nn

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cpu


### Loading a wine dataset for testing.

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html

In [3]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine.target

print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)

(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']



Creating a MLP model for the wine dataset with a hidden layer of 32 neurons

In [4]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden  = nn.Linear(input_size, hidden_size)
    self.relu    = nn.ReLU()
    self.out     = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax(dim=-1) # Softmax is used for multi-class classification probability

  def forward(self, X):
    
    feature = self.relu(self.hidden(X))
    output  = self.softmax(self.out(feature))

    return output

input_size  = data.shape[1]
hidden_size = 32
out_size    = len(wine.target_names)

net = WineClassifier(input_size, hidden_size, out_size).to(device) #cast on GPU 
print(net)

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=-1)
)


## Classification

For this use case, we will use CrossEntropyLoss loss function because it is a multi-class classification problem

Ref: https://pytorch.org/docs/stable/nn.html#crossentropyloss 


In [7]:
# Cast values as torch tensors and define loss function
criterion = nn.CrossEntropyLoss()
Xtns = torch.from_numpy(data).float()
Ytns = torch.from_numpy(target)

# Cast on GPU
criterion = criterion.to(device)
Xtns = Xtns.to(device)
Ytns = Ytns.to(device)

print(f"Xtns: {Xtns.shape}, Ytns: {Ytns.shape}\nXtns dtype: {Xtns.dtype}, Ytns dtype: {Ytns.dtype}\n")

# Define prediction
pred = net(Xtns)
print(f"pred: {pred.shape}\n")
print(f"pred[0].data: {pred[0].data}, Ytns[0].data: {Ytns[0].data}")

Xtns: torch.Size([178, 13]), Ytns: torch.Size([178])
Xtns dtype: torch.float32, Ytns dtype: torch.int64

pred: torch.Size([178, 3])

pred[0].data: tensor([1.1575e-14, 9.5790e-16, 1.0000e+00]), Ytns[0].data: 0


### Loss functions dimensions for CrossEntropy:
* prediction: ```(N, C)```
* target: ```(N,)```

In [8]:
loss = criterion(pred, Ytns)
print(loss)

tensor(1.2818, grad_fn=<NllLossBackward0>)


## Regression


### Loading a diabetes dataset for regression testing.
https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset

In [9]:
from sklearn import datasets

diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target

print(data.shape, target.shape)

print(data[14])
print(target[14])

(442, 10) (442,)
[ 4.53409833e-02 -4.46416365e-02 -2.56065715e-02 -1.25561242e-02
  1.76943802e-02 -6.12835791e-05  8.17748397e-02 -3.94933829e-02
 -3.19876395e-02 -7.56356220e-02]
118.0


Implementing DiabetesClassifier MLP

In [11]:
class DiabetesClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(DiabetesClassifier, self).__init__()

    self.hidden  = nn.Linear(input_size, hidden_size)
    self.relu    = nn.ReLU()
    self.out     = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, X):
    
    feature = self.relu(self.hidden(X))
    output  = self.softmax(self.out(feature))

    return output

input_size  = data.shape[1]
hidden_size = 32
out_size    = 1  # Progressão da diabetes

net = DiabetesClassifier(input_size, hidden_size, out_size).to(device) #cast na GPU 

### *MSELoss* mean squared error function:
Doc: https://pytorch.org/docs/stable/nn.html#mseloss

In [14]:
# Cast on GPU
criterion = nn.MSELoss().to(device)
Xtns = torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)

print(f"Xtns: {Xtns.shape}, Ytns: {Ytns.shape}\nXtns dtype: {Xtns.dtype}, Ytns dtype: {Ytns.dtype}\n")

# Define prediction
pred = net(Xtns)
print(f"pred: {pred.shape}\n")

loss = criterion(pred.squeeze(), Ytns)
print(f"loss: {loss}")

Xtns: torch.Size([442, 10]), Ytns: torch.Size([442])
Xtns dtype: torch.float32, Ytns dtype: torch.float32

pred: torch.Size([442, 1])

loss: 28771.21484375
loss: 28771.21484375


### L1 Loss Function
https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html#torch.nn.L1Loss

In [19]:
criterion = nn.L1Loss().to(device)
pred = net(Xtns)

loss = criterion(pred.squeeze(), Ytns)
print(f"loss: {loss}")

loss: 151.13348388671875


## All pytorch loss functions doc
https://pytorch.org/docs/stable/nn.html#loss-functions